In [15]:
import torch
import torchvision
trans=torchvision.transforms.Compose(
    [torchvision.transforms.PILToTensor(),
     torchvision.transforms.ConvertImageDtype(torch.float)])
train_data_transformed = torchvision.datasets.FashionMNIST(
    root="/22961", train=True, download=True,
    transform=trans)

In [16]:
from torch import nn

class MyModel(torch.nn.Module):
	def __init__(self):
		super(MyModel, self).__init__()

		self.linear1 = torch.nn.Linear(784, 10)
		self.activation = torch.nn.ReLU()
		# self.softmax = torch.nn.Softmax()
		self.logsoftmax = nn.LogSoftmax(dim=1)
	
	def forward(self, x):
		# 1. X is batch of the samples, as input
		x = torch.flatten(x)

		# 2. Split input into two tensors with same length (we can assume that the length of X is even)
		assert x.shape[0] % 2 == 0
		x1, x2 = torch.split(x, 2, dim=1)
		assert x1.size() == x2.size()

		# 3. Put two tensors into the same aggregation layer (linear, for example)
		x1 = self.linear1(x1)
		x2 = self.linear1(x2)

		# x = self.linear1(x)

		# 3. Then put into activation layer
		x1 = self.activation(x1)
		x2 = self.activation(x2)


		# 4. Concatinate two halves to create output Y
		x1 = self.softmax(x1)
		x2 = self.softmax(x2)

		Y = torch.cat((x1, x2), 0)
		# torch.concat((x1, x2), 0)
		
		return Y

# model = nn.Sequential(
#           nn.Linear(784,10),    #z
#           nn.LogSoftmax(dim=1)  #log(y)
#           )
model = MyModel()
print(model)

MyModel(
  (linear1): Linear(in_features=784, out_features=10, bias=True)
  (activation): ReLU()
  (logsoftmax): LogSoftmax(dim=1)
)


In [17]:
CE_loss=nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

In [18]:
def iterate_batch():
  imgs, labels = next(iter(train_dataloader))
  imgs = imgs.flatten(start_dim=1)
  optimizer.zero_grad()
  y_model=model(imgs)
  
  loss=CE_loss(y_model,labels)
  loss.backward()
  optimizer.step()
  
  predicted_labels = y_model.argmax(dim=1)
  acc = (predicted_labels == labels).sum()/len(labels)
  return loss.detach(), acc.detach()

In [19]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(
    train_data_transformed, batch_size=256, shuffle=True)
len(train_dataloader) #num of batches in epoch

235

In [22]:
from tqdm import tqdm
batches=len(train_dataloader)
batch_loss=torch.zeros(batches)
batch_acc=torch.zeros(batches)
for idx in tqdm(range(batches)):
    batch_loss[idx], batch_acc[idx] = iterate_batch()

  0%|          | 0/235 [00:10<?, ?it/s]


ValueError: too many values to unpack (expected 2)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(range(batches),batch_loss);
plt.title("CE loss");
plt.xlabel("Batch Number");
plt.subplot(1,2,2)
plt.plot(range(batches),batch_acc);
plt.title("Accuracy");
plt.xlabel("Batch Number");